In [27]:
from matplotlib.path import Path
import numpy as np
from bluesky.tools.aero import ft, nm, kts
from bluesky.tools.geo import qdrpos, qdrdist

In [ ]:
def invasion_circle(start_lat, start_lon, start_alt, trk, gs, vs, dtlookahead, area_args):
    """
    Predict the invasion of a circular area.
    """
    
    bottom, top, ctr_lat, ctr_lon, radius = area_args

    # Ray endpoints
    end_lat, end_lon = qdrpos(start_lat, start_lon, trk, gs * dtlookahead / nm)

    # Vectors
    startpoints = np.stack([start_lat, start_lon], axis=1)
    endpoints = np.stack([end_lat, end_lon], axis=1)
    seg_vectors = endpoints - startpoints       # Vectors from start to end point
    seg_vec_norm = np.linalg.norm(seg_vectors, axis=1)      # Norm of the vectors

    # calculate the projection (of the startpoint-center vector) vector on the ray vector
    start_center_vecs = np.array([[ctr_lat, ctr_lon]]) - startpoints
    t_close = np.einsum('ij,ij->i', seg_vectors, start_center_vecs) / seg_vec_norm**2
    t_clamped = np.clip(t_close, 0, 1)  # Clamp t to [0, 1]
    closest_points = startpoints + t_clamped[:, np.newaxis] * seg_vectors  # Closest points on the segment to the center of the area
    _, distance = qdrdist(closest_points[:, 0], closest_points[:, 1], ctr_lat, ctr_lon)
    # Check if the closest point is within the radius of the area
    inside_flag = distance < radius
    # Get t_in and t_out
    virtual_closest = startpoints + t_close[:, np.newaxis] * seg_vectors
    _, virtual_distance = qdrdist(virtual_closest[:, 0], virtual_closest[:, 1], ctr_lat, ctr_lon)
    # Use Pythagorean
    _, dist_start_virtual = qdrdist(start_lat, start_lon, virtual_closest[:, 0], virtual_closest[:, 1])
    delta_t = np.where(inside_flag, np.sqrt(radius**2 - virtual_distance**2) / dist_start_virtual * t_close, -1)
    t_in = np.where(inside_flag, t_close - delta_t, -1)
    t_out = np.where(inside_flag, t_close + delta_t, -1)
    t_in[inside_flag] = np.clip(t_in[inside_flag], 0, 1)    # Clamp to [0, 1]
    t_out[inside_flag] = np.clip(t_out[inside_flag], 0, 1)  # Clamp to [0, 1]
    # Get the altitude at t_in and t_out
    alt_in = np.where(inside_flag, (start_alt + vs * t_in * dtlookahead) / ft, -1)
    alt_out = np.where(inside_flag, (start_alt + vs * t_out * dtlookahead) / ft, -1)
    # Check if the aircraft enters or crosses the area
    below_in = alt_in[inside_flag] < bottom
    above_in = alt_in[inside_flag] > top
    below_out = alt_out[inside_flag] < bottom
    above_out = alt_out[inside_flag] > top
    not_crossing = (below_in & below_out) | (above_in & above_out)
    inside_flag[inside_flag] = ~not_crossing

    # debug
    print(alt_in, alt_out)

    return inside_flag

In [3]:
ctr_lat = 30.0
ctr_lon = 120.0
radius = 5.0
bottom = 6000
top = 10000

area_args = (bottom, top, ctr_lat, ctr_lon, radius)

ACF 1: 10NM 180DEG of center, 360/3000/250

In [71]:
start_lat = 29.833293
start_lon = 120.0
start_alt = 3000 * ft
trk = 0
gs = 250 * kts
vs = 0

In [72]:
invasion_circle(np.array([start_lat]), np.array([start_lon]), np.array([start_alt]), np.array([trk]), np.array([gs]), np.array([vs]), 600, area_args)

[3000.] [3000.]


array([False])

In [29]:
end_lat, end_lon = qdrpos(start_lat, start_lon, trk, gs * 15 / nm)

In [31]:
qdrdist(ctr_lat, ctr_lon, end_lat, end_lon)

(np.float64(180.0), np.float64(8.97040754584087))